<a href="https://colab.research.google.com/github/longevity-genie/just-agents/blob/main/examples/notebooks/02_sqlite_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building your Own SQL Agent with Just-Agents ##


This notebook provides a comprehensive guide on how to create and utilize your own custom SQL agents using [just-agents](https://github.com/longevity-genie/just-agents/) library.

It is the second tutorial in a series of tutorials, next ones are:
1. Basic agents tutorial  https://colab.research.google.com/drive/1l_pdfT0FhlzFzNV792xdiMD3xt0I660z
2. Database agent tutorial (THIS ONE) https://colab.research.google.com/drive/1FQGOfIytET5HlExxu4jdXHeOMJJDL0yg
3. Coding agent tutorial https://colab.research.google.com/drive/1CbqwSwUHir6VpWA0uKVIiqlZo6m2s6qm#scrollTo=xMh-QD4m8F7V

## Working with the Open-Genes Database

For this tutorial, we'll be interacting with the Open-Genes database. We'll be using a simplified version stored in an SQLite file. You can access the original database file at:

[https://github.com/longevity-genie/longevity_gpts/blob/main/open_genes/data/open_genes.sqlite](https://github.com/longevity-genie/longevity_gpts/blob/main/open_genes/data/open_genes.sqlite)

Let's download the database first

In [ ]:
!wget https://github.com/longevity-genie/longevity_gpts/raw/main/open_genes/data/open_genes.sqlite -O open_genes.sqlite

--2024-11-14 16:30:21--  https://github.com/longevity-genie/longevity_gpts/raw/main/open_genes/data/open_genes.sqlite
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/longevity-genie/longevity_gpts/main/open_genes/data/open_genes.sqlite [following]
--2024-11-14 16:30:21--  https://raw.githubusercontent.com/longevity-genie/longevity_gpts/main/open_genes/data/open_genes.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1150976 (1.1M) [application/octet-stream]
Saving to: ‘open_genes.sqlite’

open_genes.sqlite   100%[===================>]   1.10M  --.-KB/s    in 0.06s   

2024-11-14 16:30:21 (1

Let's test the query by outputing the tables

In [ ]:
import sqlite3

def db_query(sql: str):
    """ This function execute query on open-genes sqlite table. It returns query results. """
    conn = sqlite3.connect("open_genes.sqlite")
    cursor = conn.cursor()

    cursor.execute(sql)
    try:
        rows = cursor.fetchall()
        if rows is None or len(rows) == 0:
            conn.close()
            return ""
        names = [description[0] for description in cursor.description]
        text = "; ".join(names) + "\n"
        for row in rows:
            row = [str(i) for i in row]
            text += "; ".join(row) + "\n"
    finally:
        conn.close()

    return text

Let's first check that functions work on its own, without agent

In [ ]:
tables = db_query("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", tables)


Tables: name
lifespan_change
gene_criteria
gene_hallmarks
longevity_associations



## Making a naive sql agent

Now we will create an agent, It will handle simple tasks.

First, let's install the just-agent library with pip and set up API keys for Groq and OpenAI

In [ ]:
!pip install just-agents-core==0.5.0
!pip install just-agents-examples==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.9 MB/s eta 0:00:00


In [ ]:
import os
import getpass

def get_api_keys():
  """Gets API keys from the user."""

  print("Please enter your API keys:")
  groq_api_key = getpass.getpass("GROQ_API_KEY: ")
  openai_api_key = getpass.getpass("OPENAI_API_KEY: ")

  # Set environment variables
  os.environ["GROQ_API_KEY"] = groq_api_key
  os.environ["OPENAI_API_KEY"] = openai_api_key

  print("API keys set successfully.")

# Call the function to get API keys from the user
get_api_keys()


Please enter your API keys:
GROQ_API_KEY: ··········
OPENAI_API_KEY: ··········
API keys set successfully.


The following code just allows for user to see all comands and text outputed in the interface- it is needed to see all that comes out if the output is large.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Then we will create a simple agent that will use the db_query function as a tool and will answer questions which involve the database

Let's ask the agent some simple questions

In [ ]:
import pprint
import just_agents
from just_agents import llm_options
from just_agents.base_agent import BaseAgent




prompt = "What processes are improved in GHR knockout mice?"

session: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4oMINI,
    tools=[db_query]
)


result = session.query(prompt)

session.memory.add_on_message(lambda m: pprint.pprint(m))
result = session.query(prompt)

print("RESULT+++++++++++++++++++++++++++++++++++++++++++++++")
print(result)

{'content': 'What processes are improved in GHR knockout mice?', 'role': 'user'}
{'content': 'In GHR (Growth Hormone Receptor) knockout mice, several '
            'physiological and metabolic processes have been observed to '
            'improve or change. Here are some key areas where improvements are '
            'noted:\n'
            '\n'
            '1. **Metabolic Health**: GHR knockout mice often exhibit enhanced '
            'insulin sensitivity and improved glucose metabolism, which can '
            'lead to a lower risk of obesity and type 2 diabetes.\n'
            '\n'
            '2. **Lifespan Extension**: These mice tend to have a longer '
            'lifespan compared to their wild-type counterparts. This is '
            'associated with reduced growth hormone signaling, which is linked '
            'to various longevity pathways.\n'
            '\n'
            '3. **Reduced Inflammation**: GHR knockout mice may show lower '
            'levels of chronic infla

It got this question right, but what if we want to know "Interventions on which genes extended mice lifespan most of all?"

In [ ]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options


prompt = "Interventions on which genes extended mice lifespan most of all?"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4o
    tools=[db_query]
)


result = agent.query(prompt)

agent.memory.add_on_message(lambda m: pprint.pprint(m))
result = agent.query(prompt)

print("RESULT+++++++++++++++++++++++++++++++++++++++++++++++")
print(result)

{'content': 'Interventions on which genes extended mice lifespan most of all?',
 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"sql":"SELECT gene_symbol, '
                                           'lifespan_extension_percentage FROM '
                                           'gene_interventions ORDER BY '
                                           'lifespan_extension_percentage DESC '
                                           'LIMIT 5;"}',
                              'name': 'db_query'},
                 'id': 'call_TQeDlEiPMIcgiPNpDR55t4lk',
                 'type': 'function'}]}
{'content': 'no such table: gene_interventions',
 'name': 'db_query',
 'role': 'tool',
 'tool_call_id': 'call_TQeDlEiPMIcgiPNpDR55t4lk'}
{'content': 'I am unable to access the database table containing information '
            'on gene interventions and lifespan extension in mice. If you have '
            'access to specific data or another source, I can help inte

As you see the SQL returned zero results although we know that the data exist by browsing https://open-genes.com/ and/or sqlite file. Let's modify the system prompt to get it right

In this case it was the simple change from model OPENAI-GPT4oMINI to OPENAI-GPT4o.

But how to make it better? simply call the LLMSession with a prompt.

In [ ]:
!wget https://github.com/longevity-genie/longevity_gpts/blob/main/prompts/open_genes.txt -O open_genes.txt

--2024-11-14 16:35:59--  https://github.com/longevity-genie/longevity_gpts/blob/main/prompts/open_genes.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘open_genes.txt’

open_genes.txt          [ <=>                ] 744.54K  --.-KB/s    in 0.1s    

2024-11-14 16:36:00 (5.82 MB/s) - ‘open_genes.txt’ saved [762412]



In [ ]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options

prompt = "Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the open-genes sqlite and only for mouse"

session: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4o,
    system_prompt='open_genes.txt',
    tools=[db_query]
)


result = session.query(prompt)

session.memory.add_on_message(lambda m: pprint.pprint(m))
result = session.query(prompt)

print("RESULT+++++++++++++++++++++++++++++++++++++++++++++++")
print(result)

{'content': 'Interventions on which genes extended mice lifespan most of all? '
            'Search all the relevant tables in the open-genes sqlite and only '
            'for mouse',
 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"sql":"SELECT HGNC, '
                                           'intervention_improves, '
                                           'lifespan_percent_change_max FROM '
                                           'lifespan_change WHERE '
                                           "model_organism = 'mouse' ORDER BY "
                                           'lifespan_percent_change_max DESC '
                                           'LIMIT 5;"}',
                              'name': 'db_query'},
                 'id': 'call_CtjNg4UK9gQeKrAiPpw8z3t5',
                 'type': 'function'}]}
{'content': 'HGNC; intervention_improves; lifespan_percent_change_max\n'
            'PROP1; insulin sensitivity,INS/IGFR pathway; 

Now to check it let's call the same code but with model OPENAIgpt4omini

In [ ]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options

prompt = "Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the open-genes sqlite and only for mouse"


session: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4oMINI,
    system_prompt='open_genes.txt',
    tools=[db_query]
)


result = session.query(prompt)

session.memory.add_on_message(lambda m: pprint.pprint(m))
result = session.query(prompt)

print("RESULT+++++++++++++++++++++++++++++++++++++++++++++++")
print(result)

{'content': 'Interventions on which genes extended mice lifespan most of all? '
            'Search all the relevant tables in the open-genes sqlite and only '
            'for mouse',
 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"sql": "SELECT HGNC, '
                                           'effect_on_lifespan, '
                                           'control_lifespan_mean, '
                                           'experiment_lifespan_mean, '
                                           'lifespan_percent_change_mean FROM '
                                           'lifespan_change WHERE '
                                           "model_organism = 'mouse' AND "
                                           'effect_on_lifespan LIKE '
                                           "'%increases lifespan%' ORDER BY "
                                           'lifespan_percent_change_mean DESC '
                                           'LIMIT 

This shows that even with careful arragement prompts can still fail. It is important to check the results and stability of results before anything else.

Here are some more questions to be tested

1. What genes need to be downregulated in worms to extend their lifespan?
2. What processes are improved in GHR knockout mice?
3. Which genetic intervention led to the greatest increase in lifespan in flies?
4. To what extent did the lifespan increase in mice overexpressing VEGFA?
5. Are there any liver-specific interventions that increase lifespan in mice?
6. Which gene-longevity association is confirmed by the greatest number of studies?
7. What polymorphisms in FOXO3 are associated with human longevity?
8. In which ethnic groups was the association of the APOE gene with longevity shown?
9. Is the INS gene polymorphism associated with longevity?
10. What genes are associated with transcriptional alterations?
11. Which hallmarks are associated with the KL gene?
12. What genes change their expression with aging in humans?
13. How many genes are associated with longevity in humans?
14. What types of studies have been conducted on the IGF1R gene?
15. What evidence of the link between PTEN and aging do you know?
16. What genes are associated with both longevity and altered expression in aged humans?
17. Is the expression of the ACE2 gene altered with aging in humans?
18. Interventions on which genes extended mice lifespan most of all?
19. Which knockdowns were most lifespan extending on model animals?